In [1]:
# Initial Imports

import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
from pathlib import Path
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Import flair pre-trained sentiment model
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
# Import flair Sentence to process input text
from flair.data import Sentence
import regex as re
from sentiment_functions import clean_text, score_flair




2022-01-13 18:24:14,270 loading file C:\Users\sjufa\.flair\models\sentiment-en-mix-distillbert_4.pt
2022-01-13 18:24:21,887 loading file C:\Users\sjufa\.flair\models\sentiment-en-mix-distillbert_4.pt


In [2]:


# Making the API call to retrieve tweets 

url = "https://twitter32.p.rapidapi.com/getSearch"

# These parameters need to be set before running API call... could in the future receive inputs from user
## TODO -- if input is received, datetime will need to be formatted correctly for API call

hashtag = '$TSLA'
start_date = '2022-01-05'
end_date = '2022-01-07'

querystring = {"hashtag": hashtag, "start_date": start_date,"end_date": end_date,"lang":"en"}

headers = {
    'x-rapidapi-host': "twitter32.p.rapidapi.com",
    'x-rapidapi-key': "49e5cedc9fmsh6a2df83dacfc4c1p1c3469jsn3edb2e8cb9df"
    }

response = requests.get(url, headers=headers, params=querystring).json()
df = pd.DataFrame(response['data']['tweets']).T
df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,...,possibly_sensitive,possibly_sensitive_editable,lang,supplemental_language,self_thread,quoted_status_id,quoted_status_id_str,quoted_status_permalink,conversation_control,limited_actions
1478780165127909381,Wed Jan 05 17:27:20 +0000 2022,1478780165127909381,1478780165127909381,GM is so ahead of their schedule. \n\nThey del...,False,"[0, 99]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...","{'media': [{'id': 1478780159465566211, 'id_str...","<a href=""http://twitter.com/download/iphone"" r...",None,...,False,True,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1478739739557695493,Wed Jan 05 14:46:42 +0000 2022,1478739739557695493,1478739739557695493,My advice: Keep buying $TSLA into 4Q earnings ...,False,"[0, 271]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",None,...,NaN,NaN,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1478750908703854592,Wed Jan 05 15:31:05 +0000 2022,1478750908703854592,1478750908703854592,I don’t think you can ever own enough $TSLA sh...,False,"[0, 52]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",None,...,NaN,NaN,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1479152101582331912,Thu Jan 06 18:05:16 +0000 2022,1479152101582331912,1479152101582331912,"By the end of this year, we will look back wit...",False,"[0, 96]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",None,...,NaN,NaN,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1478706918462001156,Wed Jan 05 12:36:17 +0000 2022,1478706918462001156,1478706918462001156,Tesla Robotaxi - 'it doesn't know what to do' ...,False,"[0, 70]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...","{'media': [{'id': 1478706766418530304, 'id_str...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,False,True,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1478732517997961218,Wed Jan 05 14:18:00 +0000 2022,1478732517997961218,1478732517997961218,"Before I got #FSDbeta, I wasn't honestly sure ...",False,"[0, 250]","{'hashtags': [{'text': 'FSDbeta', 'indices': [...",NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,NaN,NaN,en,None,"{'id': 1478732517997961218, 'id_str': '1478732...",NaN,NaN,NaN,NaN,NaN
1478741185531465732,Wed Jan 05 14:52:26 +0000 2022,1478741185531465732,1478741185531465732,Germany 🇩🇪 2021 vs 2019\n(to ignore last years...,False,"[0, 253]","{'hashtags': [], 'symbols': [{'text': 'tsla', ...","{'media': [{'id': 1478738891171631107, 'id_str...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,False,True,en,None,"{'id': 1478741185531465732, 'id_str': '1478741...",NaN,NaN,NaN,NaN,NaN
1478652373929062402,Wed Jan 05 08:59:32 +0000 2022,1478652373929062402,1478652373929062402,so hyped for @elonmusk’s product roadmap updat...,False,"[0, 82]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",None,...,NaN,NaN,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1479106321441976336,Thu Jan 06 15:03:22 +0000 2022,1479106321441976336,1479106321441976336,$TSLA catalysts - 1/6\n1/ FY’22 Vol/EPS increa...,False,"[0, 280]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",None,...,NaN,NaN,en,None,NaN,NaN,NaN,NaN,NaN,NaN
1478740720706011139,Wed Jan 05 14:50:36 +0000 2022,1478740720706011139,1478740720706011139,$TSLA is green. No one move,False,"[0, 27]","{'hashtags': [], 'symbols': [{'text': 'TSLA', ...",NaN,"<a href=""http://twitter.com/download/iphone"" r...",None,...,NaN,NaN,en,None,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Parsing and formatting the dataframe to index by date created and isolating the 'full_text' and 'retweet_count' columns for sentiment analysis
df_filtered = df.loc[:,["created_at","full_text", 'retweet_count']]
df_filtered.index = pd.to_datetime(df_filtered['created_at'], utc=True)
df_filtered.index.name = 'Date'
df_filtered.drop(columns='created_at', inplace=True)
df_filtered

,full_text,retweet_count
Date,,
2022-01-05 17:27:20+00:00,GM is so ahead of their schedule. \n\nThey del...,218
2022-01-05 14:46:42+00:00,My advice: Keep buying $TSLA into 4Q earnings ...,143
2022-01-05 15:31:05+00:00,I don’t think you can ever own enough $TSLA sh...,22
2022-01-06 18:05:16+00:00,"By the end of this year, we will look back wit...",52
2022-01-05 12:36:17+00:00,Tesla Robotaxi - 'it doesn't know what to do' ...,31
2022-01-05 14:18:00+00:00,"Before I got #FSDbeta, I wasn't honestly sure ...",99
2022-01-05 14:52:26+00:00,Germany 🇩🇪 2021 vs 2019\n(to ignore last years...,81
2022-01-05 08:59:32+00:00,so hyped for @elonmusk’s product roadmap updat...,70
2022-01-06 15:03:22+00:00,$TSLA catalysts - 1/6\n1/ FY’22 Vol/EPS increa...,218


In [9]:
# We need to clean up our tweets so they can be analyzed in string format

# Create a function to clean the tweets
#def clean_text(text): 
#    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split()) 


#applying this function to Text column of our dataframe
df_filtered["full_text"] = df_filtered["full_text"].aply(clean_text)
df_filtered

AttributeError: 'Series' object has no attribute 'aply'

In [5]:
# Define a function to get Flair sentiment prediction score
'''def score_flair(text):
  sentence = Sentence(text)
  classifier.predict(sentence)
  score = sentence.labels[0].score
  value = sentence.labels[0].value
  return score, value'''

# Get sentiment score for each review
df_filtered['scores_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[0])
# Predict sentiment label for each review
df_filtered['pred_flair'] = df_filtered['full_text'].apply(lambda s: score_flair(s)[1])
# Check the distribution of the score
df_filtered['scores_flair'].describe()
# Change the label of flair prediction to 0 if negative and 1 if positive
mapping = {'NEGATIVE': 0, 'POSITIVE': 1}
df_filtered['pred_flair_numerical'] = df_filtered['pred_flair'].map(mapping)
df_filtered

,full_text,retweet_count,scores_flair,pred_flair,pred_flair_numerical
Date,,,,,
2022-01-05 17:27:20+00:00,GM is so ahead of their schedule They delivere...,218,0.991020,NEGATIVE,0
2022-01-05 14:46:42+00:00,My advice Keep buying TSLA into 4Q earnings an...,143,0.877136,POSITIVE,1
2022-01-05 15:31:05+00:00,I don t think you can ever own enough TSLA shares,22,0.982436,POSITIVE,1
2022-01-06 18:05:16+00:00,By the end of this year we will look back with...,52,0.831751,NEGATIVE,0
2022-01-05 12:36:17+00:00,Tesla Robotaxi it doesn t know what to do TSLA ht,31,0.999292,NEGATIVE,0
2022-01-05 14:18:00+00:00,Before I got FSDbeta I wasn t honestly sure TS...,99,0.995495,POSITIVE,1
2022-01-05 14:52:26+00:00,Germany 2021 vs 2019 to ignore last years lock...,81,0.994392,NEGATIVE,0
2022-01-05 08:59:32+00:00,so hyped for s product roadmap update on the Q...,70,0.999736,NEGATIVE,0
2022-01-06 15:03:22+00:00,TSLA catalysts 1 6 1 FY 22 Vol EPS increases 1...,218,0.999550,POSITIVE,1


In [6]:
#Converting the categorical variable columns into numerical with get_dummies
df_dummies = pd.get_dummies(df_filtered['pred_flair'])
# Dropping the row if the scores_flair confidence level is below 0.75
df_filtered.drop(df_filtered[df_filtered.scores_flair < 0.75].index, inplace=True)
# Creating the 'df_signals' dataframe that we will use to append to our primary dataset, using only the columns with relevant numerical values, resampled by day
df_signals = df_filtered.drop(columns=['scores_flair', 'pred_flair_numerical', 'pred_flair'])
final_df = pd.concat([df_signals, df_dummies]).resample('D').sum()
final_df

,NEGATIVE,POSITIVE
Date,,
2022-01-04 00:00:00+00:00,1.0,0.0
2022-01-05 00:00:00+00:00,9.0,5.0
2022-01-06 00:00:00+00:00,4.0,2.0


### After pulling tweet data, performing sentiment analysis using flair and then filtered and resampling the dataframe, we return a dataframe with datetime indices with two columns -- flair_negative, which is the sum of all negative tweets for the day, and flair_positive, which is the sum of all positive tweets for the day -- at some point we could think about factoring in number of retweets or something along those lines to weigh the tweets by their impact, but the results from the API call already filter by this metric, so could be superfluous.

In [7]:
df_signals

,full_text
Date,
2022-01-06 00:00:00+00:00,Following were the Top 5 tickers mentioned on r wallstreetbets TSLA AMC F SPY DD The data was fetched at 06 01 2022 00 00 02 AM UTC wallstreetbets stocks stonks trading TSLA AMC F SPY DDTSLA 15s delayed Issued Press Release on January 06 08 00 00 Could A Graphite Shortage Derail The 3 Trillion EV Boom
